# Tile Arithmetic
Rasterframes includes support for arithmetic operations across tiles, calling the function on the cells that correspond to each other and producing the result of these operations as a tile. For instance, given a tile with pixels that are all 1's and a tile whose pixels are all 2's, the result of a localAdd on those tiles would be a tile with pixel values all set to 3. This example uses `localAdd`, but `localSubtract`, `localDivide`, and `localMultiply` also exist. While not used in this example, `localAddScalar`, `localSubtractScalar`, etc can be uses to perform operations with a scalar, as opposed to the local values in another tile. 

Initial configs

In [1]:
from pyrasterframes import *
from pyrasterframes.rasterfunctions import *
import pyspark
from pyspark.sql import SparkSession
from pathlib import Path

spark = SparkSession.builder. \
    master("local[*]"). \
    appName("RasterFrames"). \
    config("spark.ui.enabled", "false"). \
    getOrCreate(). \
    withRasterFrames()
    
resource_dir = Path('./samples').resolve()

filenamePattern = "L8-B{}-Elkton-VA.tiff"

def readTiff(name):
    return resource_dir.joinpath(filenamePattern.format(name)).as_uri()

bandNumbers = range(1, 5)
bandColNames = list(map(lambda n: 'band_{}'.format(n), bandNumbers))

Once imported, join the tiles together based on a `spatialKey`.

In [2]:
from functools import reduce
joinedRF = reduce(lambda rf1, rf2: rf1.asRF().spatialJoin(rf2.drop('bounds').drop('metadata')),
                  map(lambda bf: spark.read.geotiff(bf[1]) \
                      .withColumnRenamed('tile', 'band_{}'.format(bf[0])),
                  map(lambda b: (b, readTiff(b)), bandNumbers)))

New columns are added based on the results of these local functions. `localAdd` adds corresponding cells, `localDivide` divides them, `localSubtractScalar` subtracts a scalar from every cell in the input tile, and `localMultiplyScalar` multiplies each cell by a scalar value.

In [8]:
addRF = joinedRF.withColumn("B1+B2", localAdd("band_1", "band_2")).asRF()
divideRF = joinedRF.withColumn("B1/B2", localDivide("band_1", "band_2")).asRF()

In [4]:
subScalarRF = joinedRF.withColumn("B1-100", localEqualScalar("band_1", 100)).asRF()

Py4JError: An error occurred while calling o38.localEqualScalar. Trace:
py4j.Py4JException: Method localEqualScalar([class org.apache.spark.sql.Column, class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:272)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)



In [ ]:
val subConstantRF = joinedRF.withColumn("B1 - C", localSubtract($"band_1", makeConstantTile(6, 3, 3, "IntType")))
val multScalarRF = joinedRF.withColumn("B1*2", localMultiplyScalar($"band_1", 2)).asRF

`tileMax` computes the max values of the cells in a tile, `tileSum` computes the sum of all cells, `tileMean` finds the mean value, and `tileMin` finds the minimum value.

In [ ]:
addRF.select(tileMax($"B1+B2"), tileMax($"band_1"), tileMax($"band_2")).show()

In [ ]:
divideRF.select(tileSum($"B1/B2"), tileSum($"band_1"), tileSum($"band_2")).show(1)

In [ ]:
subScalarRF.select(tileMean($"B1-100"), tileMean($"band_1")).show()

In [ ]:
multScalarRF.select(tileMin($"B1*2"), tileMin($"band_1")).show()

In [ ]:
subConstantRF.select(tileMin($"B1 - C"), tileMin($"band_1")).show()